In [1]:
%cd /content
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

/content
fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
import sys
sys.path.append('/content/yolov5')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1')

In [4]:
# 구글 드라이브의 zip 파일을 직접 /content/data 폴더에 조용히(-q) 압축 해제
!unzip -q "/content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1/tar_file/train.zip" -d /content/data
!unzip -q "/content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1/tar_file/track1_test.zip" -d /content/data

replace /content/data/__MACOSX/._track1_test? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [9]:
!rm -rf /content/data/__MACOSX

# Procedure

## Step 1 : Background Movement Refinement
1-1. 특징점 검출 : `cv2.goodFeaturesToTrack`
1-2. 특징점 추적 : `cv2.calcOpticalFlowRyrLK`
1-3. 변환 행렬 계산 + RANSAC : `cv2.estimateAffinePartial2d` + RANSAC
1-4. 영상 정렬 : 구한 행렬로 이전 프레임 비틀어서 현재와 맟주기 (`cv2.warpAffine`)

## Step 2 : Movement Mask Generation
2-1. Frame Difference : `cv2.absdiff`
2-2. 이진화 : 차분 영상에서 노이즈 제거, 확실한 움직임만 담기
2-3. 가장차리 처리 : Warping으로 생긴 검은 테두리 오인하지 않도록 마스킹

## Step 3 : Evaluate Tracking Result
3-1. ROI 추출 : SiamFC가 예측한 박스 좌표 이용, 움직임 마스크에서 해당 부분 추출
3-2. 비율 계산 : 잘라낸 박스 안에 흰 픽셀 비율 계산
3-3. 최종 판독


In [10]:
!ls /content/yolov5/Datasets/train | head

01_1751_0250-1750
01_2192_0001-1500
01_4485_1167-2666
01_4485_2985-4484
02_1610_0001_1500
02_6321_0274-2773
04_8618_1215-2714
11_1
12_1
15_1


In [11]:
import json
with open("/content/data/train/01_1751_0250-1750/IR_label.json") as f:
    data = json.load(f)
print(type(data))
print(data.keys() if isinstance(data, dict) else data[:1])

<class 'dict'>
dict_keys(['exist', 'gt_rect', 'LR', 'VE', 'OC', 'FM', 'SV', 'IC', 'DBC', 'TS'])


In [12]:
data['gt_rect'][:5]

[[306, 270, 14, 11],
 [313, 269, 14, 12],
 [320, 269, 14, 13],
 [326, 269, 14, 12],
 [326, 269, 14, 12]]

In [22]:
file_path = '/content/yolov5/Datasets/train/01_2192_0001-1500/IR_label.json'
with open(file_path, 'r') as f:
    data = json.load(f)
print(f"포함된 Key 목록 : {list(data.keys())}")
print("-" * 20)

for key in data.keys():
    length = len(data[key])
    print(f"Key : {key:10} | 데이터 개수 : {length}")
print("-" * 20)
print("첫 번째 프레임 (Index 0)의 상태 : ")
print(f"  - 존재 여부 (Exist) : {data['exist'][0]}")
print(f"  - 좌표 (gt_rect) : {data['gt_rect'][0]}")
print(f"  - 가림 여부 (OC) : {data['OC'][0]}")

포함된 Key 목록 : ['exist', 'gt_rect', 'LR', 'VE', 'OC', 'FM', 'SV', 'IC', 'DBC', 'TS']
--------------------
Key : exist      | 데이터 개수 : 1500
Key : gt_rect    | 데이터 개수 : 1500
Key : LR         | 데이터 개수 : 1500
Key : VE         | 데이터 개수 : 1500
Key : OC         | 데이터 개수 : 1500
Key : FM         | 데이터 개수 : 1500
Key : SV         | 데이터 개수 : 1500
Key : IC         | 데이터 개수 : 1500
Key : DBC        | 데이터 개수 : 1500
Key : TS         | 데이터 개수 : 1500
--------------------
첫 번째 프레임 (Index 0)의 상태 : 
  - 존재 여부 (Exist) : 1
  - 좌표 (gt_rect) : [268, 279, 29, 22]
  - 가림 여부 (OC) : 0


In [23]:
import os
drive_path = '/content/drive/MyDrive/Colab Notebooks/CVIP_LAB/UAV/Track 1'
track1_test_path = '/content/data/track1_test'

print(f"Checking directory : {track1_test_path}")

if os.path.exists(track1_test_path):
    print("Directory Exists. Listing Contents.")
    contents = os.listdir(track1_test_path)
    if contents:
        for content in contents[:10]:
            print(f"- {content}")
        if len(contents) > 10:
            print(f"... and {len(contents) - 10} more items")
        else:
            print("Directory is empty")
else:
    print("Directory does not exist.")

Checking directory : /content/data/track1_test
Directory Exists. Listing Contents.
- 20190926_103046_1_6
- 20190925_200805_1_2
- 20190926_142435_1_1
- wg2022_ir_024_split_04
- 20190925_134301_1_5
- 3700000000002_101806_4
- DJI_0051_3
- 20190926_193515_1_1
- 20190925_130434_1_1
- new40_train-new
... and 207 more items


In [27]:
from __future__ import absolute_import
import numpy as np
import cv2
from tqdm import tqdm
import json
import glob
import os

from detection_siamfc import TrackerSiamFC

def iou(bbox1, bbox2):
    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]

    (x0_1, y0_1, w1_1, h1_1) = bbox1
    (x0_2, y0_2, w1_2, h1_2) = bbox2
    x1_1 = x0_1 + w1_1
    x1_2 = x0_2 + w1_2
    y1_1 = y0_1 + h1_1
    y1_2 = y0_2 + h1_2

    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <=0:
        return 0

    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)

    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [28]:
def not_exist(pred):
    return (len(pred) == 1 and pred[0] == 0) or (len(pred) == 0)

In [29]:
def eval(out_res, label_res):
    measure_per_frame = []
    penalty_measure = []
    for _pred, _gt, _exist in zip(out_res, label_res['gt_rect'], label_res['exist']):
        # Target 존재 x : _exist==False | Target 존재 O : _exist == True
        measure_per_frame.append(not_exist(_pred) if not _exist else iou(_pred, _gt) if len(_pred) > 1 else 0)
        if _exist:
            if (len(_pred) > 1 and iou(_pred, _gt) > 1e-5):
                penalty_measure.append(0)
            else:
                penalty_measure.append(1)
    if len(measure_per_frame) == 0:
        measure_per_frame_mean = 0
    else:
        measure_per_frame_mean = np.mean(measure_per_frame)

    if len(penalty_measure) == 0:
        penalty_measure_mean = 0
    else:
        penalty_measure_mean = np.mean(penalty_measure)

    return measure_per_frame_mean - 0.2 * (penalty_measure_mean ** 0.3)

# - - -- - - 연습장

In [48]:
import torch
def check_videoID(mode='IR', visualization=False):
    assert mode in ['IR', 'RGB'], 'Only Support IR or RGB to evaluate'

    count = 0
    global drive_path
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"현재 사용 장치 : {device}")
    net_path = os.path.join(drive_path, "Baseline_code", 'model.pth')
    tracker = TrackerSiamFC(net_path = net_path)
    yolo_model = tracker.initialize_yolo()
    yolo_model.to(device)
    tracker.net.to(device)

    video_paths = glob.glob(os.path.join(os.path.join("/content/data/track1_test"), "*"))
    video_num = len(video_paths)
    output_dir = os.path.join('results', tracker.name)

    for video_id, video_path in enumerate(video_paths, start=1):
        while count != 1:
            print(f"video_id : {video_id}")
            #print(f"Video_path : {video_path}")

            frame_files = sorted([f for f in os.listdir(video_path) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))])

            # frame_path = os.path.join(video_path, frame_files[0])
            # image = cv2.imread(frame_path)
            # image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # print(image_gray.shape)

            for frame_file in frame_files:
                #print(f"frame_file : {frame_file}")
                frame_path = os.path.join(video_path, frame_file)
                print(f"frame_path  : {frame_path}")
                frame = cv2.imread(frame_path)


            count += 1


if __name__ == '__main__':
    check_videoID(mode='IR', visualization=False)

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.9.0+cpu CPU



현재 사용 장치 : cpu


Fusing layers... 
YOLOv5s_s summary: 220 layers, 7926792 parameters, 0 gradients


video_id : 1
(512, 640)


In [68]:
import torch
def check_test2(mode='IR', visualization=False):
  # Lucas-Kanade Optical Flow 파라미터 설정
    lk_params = dict(winSize=(15, 15),
                     maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    assert mode in ['IR', 'RGB'], 'Only Support IR or RGB to evaluate'

    global drive_path

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"현재 사용 장치 : {device}")
    net_path = os.path.join(drive_path, "Baseline_code", 'model.pth')
    tracker = TrackerSiamFC(net_path = net_path)
    yolo_model = tracker.initialize_yolo()
    yolo_model.to(device)
    tracker.net.to(device)

    video_paths = glob.glob(os.path.join(os.path.join('/content/data/track1_test'), "*"))
    video_num = len(video_paths)
    output_dir = os.path.join('results', tracker.name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    overall_performance = []

    for video_id, video_path in enumerate(video_paths, start=1):
        video_name = os.path.basename(video_path) # 파일 경로에서 마지막 구성 요소만 추출 \\ 제일 마지막꺼
        video_file = os.path.join(video_path, "%s.mp4"%mode)
        frame_files = sorted(
            [f for f in os.listdir(video_path) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))])

        # .txt로 굳이 변환하지 않고 json에서 읽어옴
        res_file = os.path.join(video_path, 'IR_label.json')
        with open(res_file, 'r') as f:
            label_res = json.load(f)
        if 'exist' not in label_res: # exist 정보가 없을 수 있음 :gt_rect의 길이만큼 물체가 다 존재한다고 가정
            label_res['exist'] = [1] * len(label_res['gt_rect'])

        output_file = os.path.join(output_dir, '%s.txt'% video_name)

        # 만약 추론 결과 파일(output_file) 이 이미 있다면, 모델 돌리지 않고 기존 결과를 읽어서 eval만 수행
        if os.path.exists(output_file):
            with open(output_file, 'r') as file:
                content = file.read()
                data = json.loads(content)
                out_res = data['res']

        # 추론 결과가 없으므로 처음 처리 : tracker.init - YOLO로 객체 찾거나 초기화
        else:
            frame_id = 0
            out_res = []
            pred_bbox = [0] # no prediction

            #print(frame_files[0]) # 000001.jpg

            for frame_id, frame_file in enumerate(frame_files):
                frame_path = os.path.join(video_path, frame_file)
                frame = cv2.imread(frame_path) # 이건 현재 프레임만 읽고 있는것 : Frame Difference를 YOLO에게 주려면 이전 프레임을 변수에 저장해두었다가 cv2.absdiff를 수행하는 로직 추가해야 함.

                im_vis = frame.copy()
                if len(pred_bbox) == 1: # Case 1 : 객체 찾지 못함 <- (성공 시 len==4, [x, y, w, h])
                    #print(f"[Frame {frame_id}] Tracking Failed -> Trying YOLO..")
                    pred_bbox, im_vis = tracker.init(frame, yolo_model) # detection_siamfc.py : def init
                    # frame(image)를 받아서

                    if isinstance(pred_bbox, (np.ndarray, list)):
                        pred_bbox = list(map(float, pred_bbox))
                    out_res.append(pred_bbox)

                    cv2.putText(im_vis, str(frame_id), (40, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

                    if len(pred_bbox) == 1:
                        #print(f"  ---> YOLO failed to detect anything.")
                        cv2.putText(im_vis, 'Fail to detect the UAV', (100, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    else:
                        #print(f"  ---> YOLO found UAV!")
                        pred_bbox = list(map(int, pred_bbox))
                        cv2.rectangle(im_vis, (pred_bbox[0], pred_bbox[1]), (pred_bbox[0] + pred_bbox[2], pred_bbox[1] + pred_bbox[3]), (0, 0, 255), 3)

                else: # Case 2 : 추적 중 -> SiamFC 수행
                    pred_bbox = tracker.update(frame)
                    if len(pred_bbox) == 1: # SiamFC가 놓침
                        print("SiamFC Lost Target")
                    else:
                        if frame_id == 1:
                            prev = cv2.imread(frame_path)
                            prev_frame_id = frame_id
                            prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

                        p0 = cv2.goodFeaturesToTrack(prev_gray, maxCorners=50, qualityLevel=0.03, minDistance=5, blockSize=3)

                        curr = cv2.imread(frame_path)
                        curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)

                        p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, p0, None, **lk_params)

                        print(f"prev frame id : {prev_frame_id}")
                        print(f"curr frame id : {frame_id}\n\n")

                        good_new = p1[st==1]
                        good_old = p0[st==1]
                        # print(f"good_new : {good_new}")
                        # print(f"good_old : {good_old}")

                        motion = good_new - good_old

                        prev_gray = curr_gray.copy()
                        prev_frame_id = frame_id
                        p0 = good_new.reshape(-1, 1, 2)
                        print(f"p0 : {p0}")



                    pred_bbox = tracker.update(frame) # tracking
                    # print(f"Case 2 : Frame : {frame}")
                    # print(f"frame_path : {frame_path}")
                    # print(f"frame_file : {frame_file}")
                    # print(f"frame_id : {frame_id}")

if __name__ == '__main__':
    check_test2(mode='IR', visualization=False)

YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.9.0+cpu CPU

Fusing layers... 


현재 사용 장치 : cpu


YOLOv5s_s summary: 220 layers, 7926792 parameters, 0 gradients
YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.9.0+cpu CPU



prev frame id : 1
curr frame id : 1


p0 : [[[         17          30]]

 [[         17          23]]

 [[         26          20]]

 [[         12          21]]

 [[        192          78]]

 [[         14          16]]

 [[         36          30]]

 [[        602          22]]

 [[         63          27]]

 [[         36          22]]

 [[        602          30]]

 [[         13          33]]

 [[        574          22]]

 [[         45          18]]

 [[         36          17]]

 [[        211          68]]

 [[         89          20]]

 [[         46          26]]

 [[         41          32]]

 [[         64          21]]

 [[         24          14]]

 [[        203          79]]

 [[         67          15]]

 [[        203          65]]

 [[         70          19]]

 [[        596          18]]

 [[         24          30]]

 [[        620          24]]

 [[         42          23]]

 [[        574          30]]

 [[        196          75]]

 [[         89          78]

KeyboardInterrupt: 

# - - - - - 연습장 끝

In [59]:
import torch
def main(mode='IR', visualization=False):
  # Lucas-Kanade Optical Flow 파라미터 설정
    lk_params = dict(winSize=(15, 15),
                     maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    assert mode in ['IR', 'RGB'], 'Only Support IR or RGB to evaluate'

    global drive_path

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"현재 사용 장치 : {device}")
    net_path = os.path.join(drive_path, "Baseline_code", 'model.pth')
    tracker = TrackerSiamFC(net_path = net_path)
    yolo_model = tracker.initialize_yolo()
    yolo_model.to(device)
    tracker.net.to(device)

    video_paths = glob.glob(os.path.join(os.path.join('/content/data/track1_test'), "*"))
    video_num = len(video_paths)
    output_dir = os.path.join('results', tracker.name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    overall_performance = []

    for video_id, video_path in enumerate(video_paths, start=1):
        video_name = os.path.basename(video_path) # 파일 경로에서 마지막 구성 요소만 추출 \\ 제일 마지막꺼
        video_file = os.path.join(video_path, "%s.mp4"%mode)
        frame_files = sorted(
            [f for f in os.listdir(video_path) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))])

        # .txt로 굳이 변환하지 않고 json에서 읽어옴
        res_file = os.path.join(video_path, 'IR_label.json')
        with open(res_file, 'r') as f:
            label_res = json.load(f)
        if 'exist' not in label_res: # exist 정보가 없을 수 있음 :gt_rect의 길이만큼 물체가 다 존재한다고 가정
            label_res['exist'] = [1] * len(label_res['gt_rect'])

        output_file = os.path.join(output_dir, '%s.txt'% video_name)

        # 만약 추론 결과 파일(output_file) 이 이미 있다면, 모델 돌리지 않고 기존 결과를 읽어서 eval만 수행
        if os.path.exists(output_file):
            with open(output_file, 'r') as file:
                content = file.read()
                data = json.loads(content)
                out_res = data['res']

        # 추론 결과가 없으므로 처음 처리 : tracker.init - YOLO로 객체 찾거나 초기화
        else:
            print_frame_id = 0 # enumeate(frame_files) 에서 file_id 처리해서 없애도 될듯 !! (출력용으로 그냥 둬도 괜찮을듯..?)
            out_res = []
            pred_bbox = [0] # no prediction

            for file_id, frame_file in enumerate(frame_files):
                frame_path = os.path.join(video_path, frame_file)
                frame = cv2.imread(frame_path) # 이건 현재 프레임만 읽고 있는것 : Frame Difference를 YOLO에게 주려면 이전 프레임을 변수에 저장해두었다가 cv2.absdiff를 수행하는 로직 추가해야 함.

                im_vis = frame.copy()
                if len(pred_bbox) == 1: # Case 1 : 객체 찾지 못함 <- (성공 시 len==4, [x, y, w, h])
                    #print(f"[Frame {print_frame_id}] Tracking Failed -> Trying YOLO..")
                    pred_bbox, im_vis = tracker.init(frame, yolo_model) # detection_siamfc.py : def init
                    # frame(image)를 받아서

                    if isinstance(pred_bbox, (np.ndarray, list)):
                        pred_bbox = list(map(float, pred_bbox))
                    out_res.append(pred_bbox)

                    cv2.putText(im_vis, str(print_frame_id), (40, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

                    if len(pred_bbox) == 1:
                        #print(f"  ---> YOLO failed to detect anything.")
                        cv2.putText(im_vis, 'Fail to detect the UAV', (100, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    else:
                        #print(f"  ---> YOLO found UAV!")
                        pred_bbox = list(map(int, pred_bbox))
                        cv2.rectangle(im_vis, (pred_bbox[0], pred_bbox[1]), (pred_bbox[0] + pred_bbox[2], pred_bbox[1] + pred_bbox[3]), (0, 0, 255), 3)

                else: # Case 2 : 추적 중 -> SiamFC 수행
                    pred_bbox = tracker.update(frame) # tracking


                    if len(pred_bbox) == 1: # SiamFC가 놓쳤을 때, [0] return, BB안그리기
                        out_res.append(pred_bbox)

                    else: #추적 성공
                    # =====================================================================
                        if file_id == 1:
                            prev = cv2.imread(frame_path)
                            prev_file_id = file_id
                            prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

                        # TC-Filtering은 반드시 Detection 영역 내부에서만 추출해야 함.
                        # x, y, w, h = bbox 라고 할 때, roi = prev_gray[y:y+h, x:x+w]
                        # corners = cv2.goodFeaturesToTrack(roi, ...) 로 해야 할 듯.

                        # ㄴ 그렇지만.. 드론은 너무 작은 객체이므로 detection bb 내부에서 뽑으면 안될것 같음..
                        # -> RANSAC이 드론 motion을 background motion으로 오인할 수도
                        # !! 수정 요망 !!
                        p0 = cv2.goodFeaturesToTrack(prev_gray, maxCorners=50, qualityLevel=0.03, minDistance=5, blockSize=3) # 초기 Featuer 추출

                        curr = cv2.imread(frame_path)
                        curr_gray = cv2.cvtColor(curr, cv2.COLOR_BGR2GRAY)

                        # Optical Flow
                        p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, p0, None, **lk_params)

                        good_new = p1[st == 1]
                        good_old = p0[st == 1]

                        # 살아남은 Featuer만 뽑아서 TC 계산하기
                        motion = good_new - good_old
                        # ======= TC 계산 로직 ========
                        # 여기에 코드 작성해야 함.
                        # ===========================

                        prev_gray = curr_gray.copy()
                        prev_file_id = file_id
                        p0 = good_new.shape(-1, 1, 2)

                        if p0 is None or len(p0) < 10:
                            p0 = cv2.goodFeaturesToTrack(prev_gray, maxCorners=50, qualityLevel=0.03, minDistance=5, blockSize=3)

                        # cv2.goodFeaturesToTrack() + RANSAC
                        # cv2.absdiff & 이진화 (Motion Mask 생성)
                        # pred_bbox 영역 ROI 추출 및 비율 계산

                        # =====================================================================
                        pred_bbox = list(map(int, pred_bbox))
                        out_res.append(pred_bbox)
                        cv2.rectangle(im_vis,(pred_bbox[0], pred_bbox[1]), (pred_bbox[0] + pred_bbox[2], pred_bbox[1] + pred_bbox[3]), (0, 0, 255), 3)

                    cv2.putText(im_vis, str(print_frame_id), (40, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

                if visualization:
                    cv2.imshow(video_name, im_vis)
                    cv2.waitKey(1)

                print_frame_id += 1

            if visualization:
                cv2.destroyAllWindows()
            ouptut_file = os.path.join(output_dir, "%s.txt" % video_name)

            clean_out_res = []
            for bbox in out_res:
                if(len(bbox) > 1):
                    clean_out_res.append([float(x) for x in bbox])
                else:
                    clean_out_res.append(bbox)

            with open(output_file, 'w') as f:
                json.dump({'res' : clean_out_res} ,f)
        mixed_measure = eval(out_res, label_res)
        overall_performance.append(mixed_measure)
        print("[%03d/%03d] %20s %5s Fixed Measure : %.03f" % (video_id, video_num, video_name, mode, mixed_measure))
    print("[Overall] %5s Mixed Measure: %.03f\n" % (mode, np.mean(overall_performance)))

if __name__ == '__main__':
    main(mode='IR', visualization=False)

현재 사용 장치 : cpu


YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.9.0+cpu CPU

Fusing layers... 
YOLOv5s_s summary: 220 layers, 7926792 parameters, 0 gradients
YOLOv5 🚀 v7.0-460-g3fb11111 Python-3.12.12 torch-2.9.0+cpu CPU



KeyboardInterrupt: 

In [ ]:
source_dir = os.path.join('results', 'Yolo_SiamFC')

backup_dir = os.path.join(drive_path, 'results_backup')
print(f"Backing up into Google Drive : {backup_dir}")

if not os.path.exists(backup_dir):
    shutil.copytree('results', backup_dir)
    print("Back-up finished")
else:
    print("Updating backup Folder")
    !cp -r results/* "{backup_dir}/"
    print("Update Finished")